In [1]:
import Chess
using NBInclude

In [2]:
@nbinclude "Values.ipynb"
@nbinclude "Chess.ipynb"

getNextMove (generic function with 4 methods)

In [3]:
function minimaxMax(extboard::ExtendedBoard, depth::Int64)::Int32
    legalMoves = Chess.moves(extboard.board)
    if length(legalMoves) == 0 || extboard.repetionRuleDraw
        return evaluateTerminalPositionScore(extboard.board) - depth
    elseif depth == 0
        return extboard.score
    end
    maxValue = typemin(Int32)
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        value = minimaxMin(extboard, depth - 1)
        undomove!(extboard, undo)
        maxValue = max(maxValue, value)
    end
    Chess.recycle!(legalMoves)
    return maxValue
end

minimaxMax (generic function with 1 method)

In [4]:
function minimaxMin(extboard::ExtendedBoard, depth::Int64)::Int32
    legalMoves = Chess.moves(extboard.board)
    if length(legalMoves) == 0 || extboard.repetionRuleDraw
        return evaluateTerminalPositionScore(extboard.board) - depth
    elseif depth == 0
        return extboard.score
    end
    minValue = typemax(Int32)
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        value = minimaxMax(extboard, depth - 1)
        undomove!(extboard, undo)
        minValue = min(minValue, value)
    end
    Chess.recycle!(legalMoves)
    return minValue
end

minimaxMin (generic function with 1 method)

In [5]:
function minimaxOne(extboard::ExtendedBoard, depth::Int64)::Tuple{Int32, Chess.Move}
    legalMoves = Chess.moves(extboard.board)
    sideIsWhite = Chess.sidetomove(extboard.board) == Chess.WHITE
    bestMove = legalMoves[1]
    minmaxValue = sideIsWhite ? typemin(Int32) : typemax(Int32)
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        if sideIsWhite
            value = minimaxMin(extboard, depth - 1)
            if value > minmaxValue
                bestMove = move
                println("$(value) $(bestMove)")
                minmaxValue = value
            end
        else
            value = minimaxMax(extboard, depth - 1)
            if value < minmaxValue
                bestMove = move
                println("$(value) $(bestMove)")
                minmaxValue = value
            end
        end
        undomove!(extboard, undo)
    end
    return minmaxValue, bestMove
end

minimaxOne (generic function with 1 method)

In [6]:
function minimaxAll(extboard::ExtendedBoard, depth::Int64)::Tuple{Int32, Vector{Chess.Move}}
    legalMoves = Chess.moves(extboard.board)
    sideIsWhite = Chess.sidetomove(extboard.board) == Chess.WHITE
    scoredMoves = Dict{Int32, Vector{Chess.Move}}()
    minmaxValue = sideIsWhite ? typemin(Int32) : typemax(Int32)
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        if sideIsWhite
            value = minimaxMin(extboard, depth - 1)
            minmaxValue = max(value, minmaxValue)
        else
            value = minimaxMax(extboard, depth - 1)
            minmaxValue = min(value, minmaxValue)
        end
        movesWithSameScore = get(scoredMoves, value, Chess.Move[])
        push!(movesWithSameScore, move)
        scoredMoves[value] = movesWithSameScore
        println("$(value) $(move)")
        undomove!(extboard, undo)
    end
    return minmaxValue, scoredMoves[minmaxValue]
end

minimaxAll (generic function with 1 method)